In [43]:
import pandas as pd
import numpy as np
from causaltensor.cauest.DebiasConvex import DC_PR_auto_rank

In [44]:
O = np.loadtxt("MLAB_data.txt") # California Smoke Dataset
O = O[8:, :]
O = O.T
O.shape
# treated = [38], time = [19]

(39, 31)

In [45]:
def build_baseline_M(O, treated_states, treat_start_years, type = 'control'):
    if type == 'control':
        # Create boolean mask from list of treated state indices
        treated_mask = np.zeros(O.shape[0], dtype=bool)
        treated_mask[treated_states] = True
        M = O[~treated_mask, :]
    elif type == 'pre-treatment':
        M = O[:, :min(treat_start_years)]

    return M, M.shape[0], M.shape[1]



In [46]:
M, n, T = build_baseline_M(O, [38], [19], 'control')
n, T

(38, 31)

In [47]:
# M, n, T = build_baseline_M(O, [38], [19], 'pre-treatment')
# n, T

In [48]:
# -----------------------------
# 1) Treatment pattern generators
# -----------------------------

def Z_iid(M, p_treat=0.2, rng=None):
    """
    IID treatment: each cell (i,t) independently treated with prob p_treat.
    Returns a binary mask Z ∈ {0,1}^{n×T}.
    """
    rng = np.random.default_rng(rng)
    n, T = M.shape
    Z = (rng.random((n, T)) < p_treat).astype(int)
    return Z



def Z_block(M, m1, m2, rng=None):
    """Simultaneous adoption: choose m1 random units; treat from time index m2 onward."""
    rng = np.random.default_rng(rng)
    n, T = M.shape
    Z = np.zeros_like(M, dtype=int)
    treat_units = rng.choice(n, size=m1, replace=False)
    Z[treat_units, m2:] = 1
    return Z

def Z_stagger(M, m1, min_start, rng=None):
    """Staggered adoption: choose m1 random units; each gets random start >= min_start."""
    rng = np.random.default_rng(rng)
    n, T = M.shape
    Z = np.zeros_like(M, dtype=int)
    treat_units = rng.choice(n, size=m1, replace=False)
    for i in treat_units:
        j = rng.integers(min_start, T)  # start ∈ [min_start, T-1]
        Z[i, j:] = 1
    return Z

def Z_adaptive(M, lookback_a, duration_b):
    """
    Adaptive rule (endogenous): for each unit i, scan time j.
    If M[i,j] is strictly the minimum over the previous 'a' periods
    AND there was no treatment in that lookback window,
    then mark the NEXT 'b' periods as treated.
    """
    n, T = M.shape
    Z = np.zeros((n, T), dtype=int)
    for i in range(n):
        j = 0
        while j < T:
            ok = True
            for k in range(1, lookback_a + 1):
                if j - k < 0:
                    ok = False; break
                if Z[i, j - k] == 1:
                    ok = False; break
                if M[i, j] > M[i, j - k]:
                    ok = False; break
            if ok:
                # treat NEXT b periods
                end = min(T, j + duration_b + 1)
                Z[i, j+1:end] = 1
                j += lookback_a + duration_b
            else:
                j += 1
    return Z


In [49]:
def inject_treatment_centered(M, Z, *,
                              treatment_level=0.2,
                              sigma_unit_scale=1.0,
                              sigma_time_scale=0.5,
                              rng=None):
    """
    Inject treatment with unit- and time-heterogeneity, centered over treated cells:
        O = M + (tau* + delta_i + eta_t) ∘ Z

    - tau* = mean(M) * treatment_level
    - delta_i ~ N(0, sigma_unit_scale * tau*)
    - eta_t   ~ N(0, sigma_time_scale * tau*)
    - Both delta_i and eta_t are re-centered (weighted by treated counts) so that
      the ATT over treated cells equals tau*

    Returns:
        O            : observed panel
        att_true     : ground-truth ATT over treated cells
        tau_star     : intended average effect
    """
    rng = np.random.default_rng(rng)
    n, T = M.shape

    # Base effect and heterogeneity scales
    tau_star = np.mean(M) * treatment_level
    sigma_delta = sigma_unit_scale * tau_star
    sigma_eta   = sigma_time_scale * tau_star

    # Draw heterogeneity
    delta_i = rng.normal(0.0, sigma_delta, size=n)   # unit-fixed shock
    eta_t   = rng.normal(0.0, sigma_eta,   size=T)   # time-fixed shock

    # Treated counts
    treated_per_i = Z.sum(axis=1)        # times each unit is treated
    treated_per_t = Z.sum(axis=0)        # units treated at each time
    treated_total = Z.sum()

    # Center over treated cells (if any treated)
    delta_i_ctr = delta_i.copy()
    eta_t_ctr   = eta_t.copy()
    if treated_total > 0:
        # unit-centering: weighted by #treated times per unit
        if treated_per_i.sum() > 0:
            delta_mean_treated = (delta_i * treated_per_i).sum() / treated_per_i.sum()
            delta_i_ctr -= delta_mean_treated
        # time-centering: weighted by #treated units per time
        if treated_per_t.sum() > 0:
            eta_mean_treated = (eta_t * treated_per_t).sum() / treated_per_t.sum()
            eta_t_ctr -= eta_mean_treated

    # Build treatment matrix and observed panel
    Tmat = tau_star + delta_i_ctr[:, None] + eta_t_ctr[None, :]
    O = M + Tmat * Z

    # Ground-truth ATT - will be same as tau_star after unit and time centering
    att_true = (Tmat * Z).sum() / treated_total if treated_total > 0 else 0.0
    assert np.isclose(att_true, tau_star)

    return O, tau_star


In [50]:
results = []
for t_level in [0.2, 0.1, 0.05, 0.01]:
    # IID
    Z = Z_iid(M, p_treat=t_level)
    O, tau_star = inject_treatment_centered(M, Z, treatment_level=t_level)
    M_hat, tau_hat, std = DC_PR_auto_rank(O, Z)
    print(f"Treatment Pattern: IID, Treatment level: {t_level}, tau_star: {tau_star}, tau_hat: {tau_hat}")
    results.append({
        'pattern': 'IID',
        'treatment_level': t_level,
        'tau_star': tau_star,
        'tau_hat': tau_hat,
        'error': abs(tau_star - tau_hat)/tau_star
    })

    # Block
    Z = Z_block(M, m1=10, m2=20)
    O, tau_star = inject_treatment_centered(M, Z, treatment_level=t_level)
    M_hat, tau_hat, std = DC_PR_auto_rank(O, Z)
    print(f"Treatment Pattern: Block, Treatment level: {t_level}, tau_star: {tau_star}, tau_hat: {tau_hat}")
    results.append({
        'pattern': 'Block',
        'treatment_level': t_level,
        'tau_star': tau_star,
        'tau_hat': tau_hat,
        'error': abs(tau_star - tau_hat)/tau_star
    })
    # Staggered
    Z = Z_stagger(M, m1=10, min_start=20)
    O, tau_star = inject_treatment_centered(M, Z, treatment_level=t_level)
    M_hat, tau_hat, std = DC_PR_auto_rank(O, Z)
    print(f"Treatment Pattern: Staggered, Treatment level: {t_level}, tau_star: {tau_star}, tau_hat: {tau_hat}")
    results.append({
        'pattern': 'Staggered',
        'treatment_level': t_level,
        'tau_star': tau_star,
        'tau_hat': tau_hat,
        'error': abs(tau_star - tau_hat)/tau_star
    })
    # Adaptive
    Z = Z_adaptive(M, lookback_a=5, duration_b=10)
    O, tau_star = inject_treatment_centered(M, Z, treatment_level=t_level)
    M_hat, tau_hat, std = DC_PR_auto_rank(O, Z)
    print(f"Treatment Pattern: Adaptive, Treatment level: {t_level}, tau_star: {tau_star}, tau_hat: {tau_hat}")
    results.append({
        'pattern': 'Adaptive',
        'treatment_level': t_level,
        'tau_star': tau_star,
        'tau_hat': tau_hat,
        'error': abs(tau_star - tau_hat)/tau_star
    })

results_df = pd.DataFrame(results)




Treatment Pattern: IID, Treatment level: 0.2, tau_star: 23.906570454770794, tau_hat: 28.91367029811769
Treatment Pattern: Block, Treatment level: 0.2, tau_star: 23.906570454770794, tau_hat: 21.271803457205763
Treatment Pattern: Staggered, Treatment level: 0.2, tau_star: 23.906570454770794, tau_hat: 25.36495720836242
Treatment Pattern: Adaptive, Treatment level: 0.2, tau_star: 23.906570454770794, tau_hat: 25.95703694394874
Treatment Pattern: IID, Treatment level: 0.1, tau_star: 11.953285227385397, tau_hat: 11.78316149615385
Treatment Pattern: Block, Treatment level: 0.1, tau_star: 11.953285227385397, tau_hat: 11.993071989097777
Treatment Pattern: Staggered, Treatment level: 0.1, tau_star: 11.953285227385397, tau_hat: 8.586481063079008
Treatment Pattern: Adaptive, Treatment level: 0.1, tau_star: 11.953285227385397, tau_hat: 15.400865955083063
Treatment Pattern: IID, Treatment level: 0.05, tau_star: 5.9766426136926984, tau_hat: 5.892715533152063
Treatment Pattern: Block, Treatment level: 

In [51]:
results_df

,pattern,treatment_level,tau_star,tau_hat,error
0,IID,0.20,23.906570,28.913670,0.209445
1,Block,0.20,23.906570,21.271803,0.110211
2,Staggered,0.20,23.906570,25.364957,0.061004
3,Adaptive,0.20,23.906570,25.957037,0.085770
4,IID,0.10,11.953285,11.783161,0.014232
5,Block,0.10,11.953285,11.993072,0.003329
6,Staggered,0.10,11.953285,8.586481,0.281664
7,Adaptive,0.10,11.953285,15.400866,0.288421
8,IID,0.05,5.976643,5.892716,0.014043
9,Block,0.05,5.976643,1.897685,0.682483
